<h1>Вступительное задание для вакансии Junior Python Programmer (стажер)</h1>

3. На языке Python реализовать функцию, которая быстрее всего (по процессорным тикам) отсортирует данный ей массив чисел. Массив может быть любого размера со случайным порядком чисел (в том числе и отсортированным). Объяснить почему вы считаете, что функция соответствует заданным критериям.

<h2>Решение</h2>

Наиболее популярным алгоритмом сортировки на данный момент является Timsort. Именно он реализован в методе sort() в Python. Он стабилен для различных типов массивов (к примеру quicksort деградирует до O(n<sup>2</sup>) в специально подобранных списках), поэтому для такой постановки он подойдёт лучше всего.
С другой стороны, массив может быть любого размера, а если быть точнее, то он может храниться где-нибудь на hdd или sdd и весить десятки гигабайт и не влезать в RAM. ~~Тут лучше использовать С/С++.~~

Такой массив можно разбить на несколько частей, отсортировать их, а затем объединить эти части.

In [9]:
import glob
from os import remove
from heapq import merge

class large_list:

    def __init__(self, path):
        self.path = path

    def __chunking(self, chunksize):
        lines = []
        fid = 0
        with open(self.path, 'r') as f_in:
            f_out = open('chunk_{}.tsv'.format(fid), 'w')
            for line_num, line in enumerate(f_in, 1):
                lines.append(line)
                if not line_num % chunksize:
                    lines.sort(key=int)
                    f_out.writelines(lines)
                    f_out.close()
                    lines = []
                    fid += 1
                    f_out = open('chunk_{}.tsv'.format(fid), 'w')

            # последний кусок большого массива может быть меньше chunksize
            if lines:
                lines.sort()
                f_out.writelines(lines)
                f_out.close()
                lines = []

    
    def __rm_chunkfiles(self):
        path = "chunk_*.tsv"
        for filename in glob.glob(path):
            remove(filename)

    def sort(self, chunksize):
        self.__chunking(chunksize)
        chunks = []
        path = "chunk_*.tsv"
        for filename in glob.glob(path):
            chunks += [open(filename, 'r')]
        
        with open('sorted.tsv', 'w') as f_out:
            f_out.writelines(merge(*chunks, key=int))
        
        self.__rm_chunkfiles()

Проверим методы класса на файле, содержащем рандомные числа.

In [10]:
from random import randint
row = '{}\n'
with open('large_file.tsv', 'w') as f_out:
    for i in range(300_000):
        f_out.write(row.format(randint(1, 2147483647)))
f_out.close()

In [11]:
lfile = large_list("large_file.tsv")
lfile.sort(10000)